In [10]:
#### UMSIvsTOTAGO  ####
## Overpass Explorer ##
###### Version 1 ######
import pandas as pd
import requests
import requests_cache
import json
import os
import pickle

In [2]:
## HELPFUL UTILITY FUNCTIONS ##
import util
from pprint import pprint

In [3]:
DEBUG_MODE = False
# WAYLIMIT = 100
requests_cache.install_cache('demo_cache')

In [9]:
cache_fname = "geocache.json" #Create and open a cache file
try:
    cacheFile = open(cache_fname, 'r')
    saved_cache = pickle.load(myCacheFile)
    cacheFile.close()
except:
    saved_cache = {}

In [35]:
"""def get_with_caching(query, cacheDict, cache_fname):
	query = query
	if query in cacheDict:
		return cacheDict[query]
	else:
		myPckg = {'data':query}
		outs = requests.get('https://overpass-api.de/api/interpreter', params = myPckg)
		#cacheDict[query] = outs.text
		geoJ = json.loads(outs)
		geoJelements = geoJ['elements']
		geoJelements_df = pd.DataFrame(geoJelements)
		myCacheFile = open(cache_fname, "wb")
		pickle.dump(cacheDict, myCacheFile)
		myCacheFile.close()
		return geoJelements_df"""

In [ ]:
"""df_cache = "dfcache.json" #Create and open a cache file
try:
    df_cacheFile = open(df_cache, 'r')
    saved_df_cache = pickle.load(df_myCacheFile)
    df_cacheFile.close()
except:
    saved_df_cache = {}"""

In [4]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 10)

In [5]:
## IN: nothing, yet, will add parameters
## Out: nodeID, begin_lat, begin_lon, end_lat, end_lon
def queryToDf():
	
	'''TODO: implement in util.py'''
	# areaid = util.getArea(area)		
	# areaid = '3600165789'
	query = '[out:json][timeout:25]; area(3600165789)->.searchArea; (way["highway"~"path|footway|cycleway|bridleway"]\
	["name"~"trail|Trail|Hiking|hiking"](area.searchArea);<;);(._;>;);out;'
	# query = '[out:json][timeout:25];area(3600165789)->.searchArea;relation["route"="hiking"](area.searchArea);(._;>;);out;'
	# query = '[out:json][timeout:25];relation["route"="hiking"](46.561516046166,-87.437782287598,46.582255876979,-87.39284992218);(._;>;);out;'
	pckg = {'data':query}
	outs = requests.get('https://overpass-api.de/api/interpreter', params=pckg)
	geoJ = json.loads(outs.text)
	geoJelements = geoJ['elements']
	geoJelements_df = pd.DataFrame(geoJelements)
	return geoJelements_df

In [6]:
## To be applied to DF:
##  IN: dataframe row object 
## OUT: list of nodes in relation
def transform_members(c, way_df, nod_df):
	ways = []
	nodes = []
	errors = []
	badways = []
	for way in c['members']:
		if way['type'] == "node":
			badways.append((c['id'], way))
			break
		elif way['type'] == "way":
			try:
				w = way_df.loc[way_df['id'] == way['ref']]
				way_id = 0

				way_tags = list(dict(w['tags']).values())
				role = way['role']
				try: 
					tags = list(dict(w['tags']).values())[0]
				except Exception:
					"this only happens if there aren't tags to begin with"
					tags = []
				way = {'id':way_id, 'tags':tags, 'role':role}
			# way = {'id':int(w['id']), 'tags':list(dict(w['tags']).values())[0]}
				ways.append(way)

				for node in list(w['nodes'])[0]:
					n = nod_df.loc[nod_df['id'] == node]
					node = {'id':n['id'], 'lat':n['lat'], 'lon':n['lon']}
					nodes.append(node)

			except Exception as e:
				errors.append((e, c))
				print("way error: ", e, " on: \n")
				print(c)
				print("*************************")
	c['ways'] = ways
	c['nodes'] = nodes
	return c

In [40]:
query = '[out:json][timeout:25]; area(3600165789)->.searchArea; (way["highway"~"path|footway|cycleway|bridleway"]\
["name"~"trail|Trail|Hiking|hiking"](area.searchArea);<;);(._;>;);out;'

In [44]:
def get_with_caching(query, cacheDict, cache_fname):
	query = query
	if query in cacheDict:
		return cacheDict[query]
	else:
		myPckg = {'data':query}
		outs = requests.get('https://overpass-api.de/api/interpreter', params = myPckg)
		#cacheDict[query] = outs.text
		geoJ = json.loads(outs.text)
		geoJelements = geoJ['elements']
		geoJelements_df = pd.DataFrame(geoJelements)
		dfs = [x for _, x in geoJelements_df.groupby('type')]
		nod_df = dfs[0]
		rel_df = dfs[1]
		way_df = dfs[2]
		print("found ",len(rel_df), " trails!")
		# 3. new df from rel_df, including column containing ways, nodes + their respective data
		trail_df = rel_df.apply(transform_members, args=(way_df, nod_df), axis=1)
		trail_df = trail_df.dropna(axis=1, how='all')
		cacheDict[query] = trail_df
		#print(trail_df.head())
		myCacheFile = open(cache_fname, "wb")
		pickle.dump(cacheDict, myCacheFile)
		myCacheFile.close()
		return trail_df

In [45]:
def main():
    trail_df = get_with_caching(query, saved_cache, cache_fname)
    print(trail_df.head())

In [46]:
"""def queryToDfCached():
	query = '[out:json][timeout:25]; area(3600165789)->.searchArea; (way["highway"~"path|footway|cycleway|bridleway"]\
	["name"~"trail|Trail|Hiking|hiking"](area.searchArea);<;);(._;>;);out;'
	# query = '[out:json][timeout:25];area(3600165789)->.searchArea;relation["route"="hiking"](area.searchArea);(._;>;);out;'
	# query = '[out:json][timeout:25];relation["route"="hiking"](46.561516046166,-87.437782287598,46.582255876979,-87.39284992218);(._;>;);out;'
	pckg = {'data':query}
	outs = get_with_caching(query, saved_cache, cache_fname)
	geoJ = json.loads(outs)
	geoJelements = geoJ['elements']
	geoJelements_df = pd.DataFrame(geoJelements)"""


'def queryToDfCached():\n\tquery = \'[out:json][timeout:25]; area(3600165789)->.searchArea; (way["highway"~"path|footway|cycleway|bridleway"]\t["name"~"trail|Trail|Hiking|hiking"](area.searchArea);<;);(._;>;);out;\'\n\t# query = \'[out:json][timeout:25];area(3600165789)->.searchArea;relation["route"="hiking"](area.searchArea);(._;>;);out;\'\n\t# query = \'[out:json][timeout:25];relation["route"="hiking"](46.561516046166,-87.437782287598,46.582255876979,-87.39284992218);(._;>;);out;\'\n\tpckg = {\'data\':query}\n\touts = get_with_caching(query, saved_cache, cache_fname)\n\tgeoJ = json.loads(outs)\n\tgeoJelements = geoJ[\'elements\']\n\tgeoJelements_df = pd.DataFrame(geoJelements)'

In [47]:
"""def main():
	# 1. query OSM and send results to df
    geoJelements_df = queryToDfCached()

	# 2. split into 3 dfs by type 
	dfs = [x for _, x in geoJelements_df.groupby('type')]
	nod_df = dfs[0]
	rel_df = dfs[1]
	way_df = dfs[2]
	print("found ",len(rel_df), " trails!")
	# 3. new df from rel_df, including column containing ways, nodes + their respective data
	trail_df = rel_df.apply(transform_members, args=(way_df, nod_df), axis=1)
	trail_df = trail_df.dropna(axis=1, how='all')
	print(trail_df.head())"""

'def main():\n\t# 1. query OSM and send results to df\n    geoJelements_df = queryToDfCached()\n\n\t# 2. split into 3 dfs by type \n\tdfs = [x for _, x in geoJelements_df.groupby(\'type\')]\n\tnod_df = dfs[0]\n\trel_df = dfs[1]\n\tway_df = dfs[2]\n\tprint("found ",len(rel_df), " trails!")\n\t# 3. new df from rel_df, including column containing ways, nodes + their respective data\n\ttrail_df = rel_df.apply(transform_members, args=(way_df, nod_df), axis=1)\n\ttrail_df = trail_df.dropna(axis=1, how=\'all\')\n\tprint(trail_df.head())'

In [50]:
if __name__ == '__main__':
	main()

            id  \
176494  22308    
176495  48886    
176496  50559    
176497  195704   
176498  226171   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           